In [1]:
import pandas as pd

/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_41713/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
from mb_functions import *

In [7]:
SPOTIPY_CLIENT_ID= 'a5d9928d7d874cebafc8a8a632277854'
SPOTIPY_CLIENT_SECRET= 'a3341b047daa4007a5bc3ad2fc9413c7'
SPOTIPY_REDIRECT_URI = 'http://127.0.0.1:9090'
SCOPE = 'user-top-read user-library-read playlist-modify-public'

In [21]:
auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_REDIRECT_URI, scope=SCOPE)

In [22]:
token = auth_manager.get_access_token()['access_token']

/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_41713/1840087059.py:1: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = auth_manager.get_access_token()['access_token']


In [4]:
df = pd.read_csv('assets/tracks_vibes.csv')

In [25]:
import pandas as pd
import requests

def get_audio_features(track_ids, access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    track_data = []

    # Retrieve audio features in batches of 100
    count = 1
    for i in range(0, len(track_ids), 100):
        print(count)
        count+=1
        batch_track_ids = track_ids[i:i+100]
        audio_features_url = f'https://api.spotify.com/v1/audio-features/?ids={",".join(batch_track_ids)}'
        response = requests.get(audio_features_url, headers=headers)

        # Check if the response was successful
        if response.status_code == 200:
            audio_features_data = response.json()
            if 'audio_features' in audio_features_data:
                audio_features = audio_features_data['audio_features']
                # Add track information and audio features to the data frame
                for features in audio_features:
                    if features:
                        track_info = {
                            'track_id': features['id'],
                            'Danceability': features['danceability'],
                            'Energy': features['energy'],
                            'Loudness': features['loudness'],
                            'Speechiness': features['speechiness'],
                            'Acousticness': features['acousticness'],
                            'Instrumentalness': features['instrumentalness'],
                            'Liveness': features['liveness'],
                            'Valence': features['valence'],
                            'Tempo': features['tempo'],
                            'Duration_ms': features['duration_ms'],
                            'Time_Signature': features['time_signature']
                        }
                        track_data.append(track_info)
            else:
                print("The 'audio_features' key was not found in the response.")
        else:
            print(f"Failed to fetch audio features. Status code: {response.status_code}, Response: {response.text}")

    return pd.DataFrame(track_data)


In [26]:
track_ids = df['track_id'].tolist()

# Get audio features DataFrame
audio_features_df = get_audio_features(track_ids, token)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252


In [34]:
# Merge the audio features back into the original DataFrame on 'track_id'
merged_df = pd.merge(df, audio_features_df, on='track_id', how='left')

In [38]:
merged_df = merged_df.drop_duplicates().reset_index(drop = True)

In [49]:
def get_artist_info(artist_ids, access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    artist_data = []

    # Retrieve artist details in batches of 50
    for i in range(0, len(artist_ids), 50):
        batch_artist_ids = artist_ids[i:i+50]
        artists_url = f'https://api.spotify.com/v1/artists?ids={",".join(batch_artist_ids)}'
        response = requests.get(artists_url, headers=headers)
        
        if response.status_code == 200:
            artists_response = response.json()
            if 'artists' in artists_response:
                artists = artists_response['artists']
                for artist in artists:
                    if artist:  # Ensure artist is not None
                        artist_info = {
                            'Artist ID': artist['id'],
                            'Artist Followers': artist['followers']['total'],
                            'Artist Genres': ', '.join(artist['genres']),
                            'Artist Popularity': artist['popularity']
                        }
                        artist_data.append(artist_info)
            else:
                print("The 'artists' key was not found in the response.")
        else:
            print(f"Failed to fetch artist details. Status code: {response.status_code}, Response: {response.text}")
    
    return pd.DataFrame(artist_data)

def fetch_artist_ids_and_mapping_from_tracks(track_ids, access_token):
    headers = {'Authorization': f'Bearer {access_token}'}
    artist_ids = set()
    track_artist_mapping = []

    for i in range(0, len(track_ids), 50):
        batch_track_ids = track_ids[i:i+50]
        tracks_url = f'https://api.spotify.com/v1/tracks?ids={",".join(batch_track_ids)}'
        response = requests.get(tracks_url, headers=headers)
        
        if response.status_code == 200:
            tracks_data = response.json()['tracks']
            for track in tracks_data:
                if track:  # Ensure track is not None
                    for artist in track['artists']:
                        artist_ids.add(artist['id'])
                        track_artist_mapping.append({'track_id': track['id'], 'Artist ID': artist['id']})
        else:
            print(f"Failed to fetch track details for batch. Status code: {response.status_code}, Response: {response.text}")
    
    return list(artist_ids), pd.DataFrame(track_artist_mapping)

def get_artist_info_and_merge(df, access_token):
    track_ids = df['track_id'].unique().tolist()
    artist_ids, track_artist_mapping_df = fetch_artist_ids_and_mapping_from_tracks(track_ids, access_token)
    
    # Fetch artist details
    artist_df = get_artist_info(artist_ids, access_token)
    
    # Merge the track_artist_mapping_df with artist_df to include artist details
    detailed_track_artist_df = pd.merge(track_artist_mapping_df, artist_df, on='Artist ID', how='left')
    
    # Merge this detailed artist information back into the original DataFrame
    # This assumes your original DataFrame has a 'track_id' column to merge on
    merged_df = pd.merge(df, detailed_track_artist_df, on='track_id', how='left')
    
    return merged_df



In [50]:
df_audio_artists = get_artist_info_and_merge(merged_df,token)

In [56]:
df_audio_artists_clean = df_audio_artists.drop_duplicates(subset = ['search_id','track_id']).reset_index(drop = True)

In [77]:
df = df_audio_artists_clean.copy()

In [78]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA

# Assuming 'df' is your DataFrame
# Split 'Artist Genres' into a list of genres
df['Artist Genres'] = df['Artist Genres'].str.split(', ').apply(lambda x: x if isinstance(x, list) else [])

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform 'Artist Genres' into a binary matrix
genres_matrix = mlb.fit_transform(df['Artist Genres'])

# Create a sparse DataFrame with one column for each genre
sparse_genres_df = pd.DataFrame(genres_matrix, columns=mlb.classes_)

# Adding the 'search_id' or 'track_id' to the sparse dataframe for later merging
sparse_genres_df['track_id'] = df['track_id']

In [79]:
# Applying PCA to reduce dimensionality to 5 principal components
pca = PCA(n_components=5)
reduced_data = pca.fit_transform(sparse_genres_df.drop('track_id', axis=1))

# Creating a DataFrame for the PCA-reduced data
pca_genres_df = pd.DataFrame(reduced_data, columns=[f'PC{i+1}' for i in range(5)])

# Adding the 'track_id' back to the PCA-reduced DataFrame for identification
pca_genres_df['track_id'] = df['track_id']

# If you need to merge this back with the original DataFrame
# This will add the PCA components as new columns in your original DataFrame


In [80]:
df_merged = pd.merge(df, pca_genres_df, on='track_id', how='left')

In [83]:
df_final = df_merged.drop_duplicates(subset = ['search_id','track_id']).reset_index(drop = True)

In [87]:
df_final.to_csv('assets/df_pca.csv' ,index = False)